In [44]:
from __future__ import division
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
from sklearn.decomposition import PCA
import pandas as pd
import math
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score

import sys; sys.path.append('../Utils')
from utils import Utils

In [45]:
ut = Utils()
X, Y = ut.get_matrixes()

In [46]:
def prueba_svm(ker='linear',conf=0.001,gam=0.0):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = []
    EficienciaVal = []
    skf = StratifiedKFold(n_splits=Folds)
    porc_vectores = []
    f1 = []
    gmean = []
    for train, test in skf.split(X, Y):
        Xtrain, Ytrain, Xtest, Ytest = ut.get_training_test(X, Y, train, test)
        
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if ker == 'linear':
            modelo = SVC(kernel =ker, C=conf)
        else:
            modelo = SVC(kernel=ker, C= conf, gamma= gam)
        modelo.fit(Xtrain,Ytrain)

        f1_score_current, gmean_current, eficiencia_train_current, eficiencia_test_current = ut.get_metrics(modelo, Xtrain, Xtest, Ytrain, Ytest)
        
        #Evaluamos las predicciones del modelo con los datos de test      
        gmean.append(gmean_current)
        f1.append(f1_score_current)
        EficienciaTrain.append(eficiencia_train_current)
        EficienciaVal.append(eficiencia_test_current)
        porc_vectores.append(len(modelo.support_vectors_)/len(Xtrain))

    f1_m = np.mean(f1)
    f1_ic = np.std(f1)
    gmean = np.mean(gmean)
    gmean_ic = np.std(gmean)
    efi_train = np.mean(EficienciaTrain)
    efi_train_ic = np.std(EficienciaTrain)
    efi_test = np.mean(EficienciaVal)
    efi_test_ic = np.std(EficienciaVal)

    print('f1 = ' + str(f1_m) + '+-' + str(f1_ic))
    print('gmean = ' + str(gmean) + '+-' + str(gmean_ic))
    print('Eficiencia Train= ' + str(efi_train) + '+-' + str(efi_train_ic))
    print('Eficiencia Test= ' + str(efi_test) + '+-' + str(efi_test_ic))
        
    return f1_m, f1_ic, gmean, gmean_ic,efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectores

In [47]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kern' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})

df_types["f1"] = ""
df_types["IC f1"] = ""
df_types["gmean"] = ""
df_types["IC gmean"] = ""
df_types["Eficiencia Train"] = ""
df_types["IC Eficiencia Train"] = ""
df_types["Eficiencia Test"] = ""
df_types["IC Eficiencia Test"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kern','C','gamma'], inplace=True)

In [48]:
j = 0
for i in df_types.index:
    print(i)
    kernel = i[0]
    c = i[1]
    gamma = i[2]
    f1_m, f1_ic, gmean, gmean_ic, efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectore = prueba_svm(ker=kernel,conf=c,gam=gamma)
    df_types["f1"] = str(f1_m)
    df_types["IC f1"] = str(f1_ic)
    df_types["gmean"] = str(gmean)
    df_types["IC gmean"] = str(gmean_ic)
    df_types["Eficiencia Train"] = efi_train
    df_types["IC Eficiencia Train"] = efi_train_ic
    df_types["Eficiencia Test"] = efi_test
    df_types["IC Eficiencia Test"] = efi_test_ic
    df_types["% de Vectores de Soporte"] = str(porc_vectore)
    j = j+1

('linear', 0.001, 0.0)
f1 = 0.5802298448636799+-0.14128353878030434
gmean = 0.7069487321952701+-0.0
Eficiencia Train= 0.6170598736971357+-0.07425919864572549
Eficiencia Test= 0.6189042588115916+-0.10275617353048849
('linear', 0.01, 0.0)
f1 = 0.7818783851459411+-0.03539684584435777
gmean = 0.8334832366290696+-0.0
Eficiencia Train= 0.7993289961983352+-0.007120904592584718
Eficiencia Test= 0.780459728726379+-0.03251717343629171
('linear', 0.1, 0.0)
f1 = 0.8306485353087093+-0.04164515722162036
gmean = 0.8715506901078198+-0.0
Eficiencia Train= 0.8561265710321619+-0.0033681054082311643
Eficiencia Test= 0.8302111758607221+-0.039371808182593344
('linear', 1.0, 0.0)
f1 = 0.8515561007312563+-0.034859706254855675
gmean = 0.8873902610779109+-0.0
Eficiencia Train= 0.8739176095816841+-0.0047477593461559674
Eficiencia Test= 0.8509591117235723+-0.033808134639144345
('linear', 10.0, 0.0)
f1 = 0.8700425955055768+-0.02332855857415722
gmean = 0.9015052428762647+-0.0
Eficiencia Train= 0.8924475341686784+-0

In [49]:
def train_svm(kernel, C, gamma):
    Folds = 4
    random.seed(19680801)
    f1 = []
    gmean = []
    skf = StratifiedKFold(n_splits=Folds)
    EficienciaTrain = []
    EficienciaVal = []
    
    porc_vectores = []
    models = []
    for train, test in skf.split(X, Y):
        Xtrain, Ytrain, Xtest, Ytest = ut.get_training_test(X, Y, train, test)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        modelo = SVC(C=C,kernel=kernel, gamma=gamma, probability=True)
        modelo.fit(Xtrain, Ytrain)
        models.append(modelo)
 
        f1_score_current, gmean_current, eficiencia_train_current, eficiencia_test_current = ut.get_metrics(model=modelo, X_train=Xtrain, X_test, y_train=Ytrain, Ytest=Ytest)
        #Evaluamos las predicciones del modelo con los datos de test      
        gmean.append(gmean_current)
        f1.append(f1_score_current)
        EficienciaTrain.append(eficiencia_train_current)
        EficienciaVal.append(eficiencia_test_current)
        porc_vectores.append(len(modelo.support_vectors_)/len(Xtrain))
        
    f1_m = np.mean(f1)
    f1_ic = np.std(f1)
    gmean = np.mean(gmean)
    gmean_ic = np.std(gmean)
    efi_train = np.mean(EficienciaTrain)
    efi_train_ic = np.std(EficienciaTrain)
    efi_test = np.mean(EficienciaVal)
    efi_test_ic = np.std(EficienciaVal)

    print('f1 = ' + str(f1_m) + '+-' + str(f1_ic))
    print('gmean = ' + str(gmean) + '+-' + str(gmean_ic))
    print('Eficiencia Train= ' + str(efi_train) + '+-' + str(efi_train_ic))
    print('Eficiencia Test= ' + str(efi_test) + '+-' + str(efi_test_ic))
        
    return f1_m, f1_ic, gmean, gmean_ic,efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectores

SyntaxError: positional argument follows keyword argument (<ipython-input-49-be6fae93a49e>, line 20)

In [ ]:
df_types = pd.DataFrame({
    'Kern' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})


In [ ]:
import qgrid
randn = np.random.randn

f1_ = []
stdf1_ = []
gmean_ = []
stdgmean_ = []
efi_train_ = []
std_efi_train_ = []
efi_test_ = []
std_efi_test_ = []
porc_vect = []
for e, i, j in zip(df_types['Kern'], df_types['C'], df_types['gamma']):
    if(e == 'lineal'):
        f1_m, f1_ic, gmean, gmean_ic,efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectores = train_svm("linear", i, "auto")
    else:
        f1_m, f1_ic, gmean, gmean_ic, efi_train, efi_train_ic, efi_test, efi_test_ic, porc_vectores = train_svm(e, i, j)
    f1_.append(f1_m)
    stdf1_.append(f1_ic)
    gmean_.append(gmean)
    stdgmean_.append(gmean_ic)
    efi_train_.append(efi_train)
    std_efi_train_.append(efi_train_ic)
    efi_test_.append(efi_test)
    std_efi_test_.append(efi_test_ic)
    porc_vect.append(porc_vectores)

df_types["f1"] = f1_
df_types["Intervalo de confianza f1"] = stdf1_
df_types["G-mean"] = gmean_
df_types["Intervalo de confianza G-mean"] = stdgmean_
df_types["Eficiencia Train"] = efi_train_
df_types["IC Eficiencia Train"] = std_efi_train_
df_types["Eficiencia Test"] = efi_test_
df_types["IC Eficiencia Test"] = std_efi_test_
df_types["% de Vectores de Soporte"] = porc_vect

In [38]:
df_types

,Kern,C,gamma,f1,Intervalo de confianza f1,G-mean,Intervalo de confianza G-mean,Eficiencia Train,IC Eficiencia Train,Eficiencia Test,IC Eficiencia Test
0,lineal,0.001,0.00,0.580230,0.141284,0.706949,0.0,0.617060,0.074259,0.618904,0.102756
1,lineal,0.010,0.00,0.781878,0.035397,0.833483,0.0,0.799329,0.007121,0.780460,0.032517
2,lineal,0.100,0.00,0.830649,0.041645,0.871551,0.0,0.856127,0.003368,0.830211,0.039372
3,lineal,1.000,0.00,0.851556,0.034860,0.887390,0.0,0.873918,0.004748,0.850959,0.033808
4,lineal,10.000,0.00,0.870043,0.023329,0.901505,0.0,0.892448,0.003997,0.869490,0.023127
5,lineal,100.000,0.00,0.868710,0.027538,0.900580,0.0,0.894797,0.004220,0.868284,0.027403
6,rbf,0.001,0.01,0.315290,0.258237,0.557719,0.0,0.431574,0.169848,0.439412,0.184664
7,rbf,0.001,0.10,0.315414,0.258452,0.557874,0.0,0.433857,0.173802,0.439613,0.185013
8,rbf,0.001,1.00,0.313692,0.255468,0.556788,0.0,0.436745,0.178803,0.438204,0.182572
9,rbf,0.010,0.01,0.315290,0.258237,0.557719,0.0,0.431574,0.169848,0.439412,0.184664
